## OCR

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import uuid
import logging
import getpass
import numpy as np
import cv2
import os
import gc
import time
# import imageio
import random
import shutil
import copy
import glob
import json
import shutil
import pickle
import traceback
import zipfile
from collections import Counter
from pprint import pprint as print

In [3]:
import cv2
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm, tqdm_notebook
from PIL import Image, ImageDraw, ExifTags
# from shapely.geometry import Polygon
# import imgaug
# import imgaug as ia
# import imgaug.augmenters as iaa
# import skimage
# from skimage import io, color
import torch
from torch.utils.data import DataLoader

In [4]:
from test_api import CLI, CVAT_API_V1, ResourceType

In [5]:
def imshow(img, figsize=(16, 16)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img)
    
def remove_all_content(fd):
    shutil.rmtree(fd)
    os.mkdir(fd)
    
def get_cookies():
    login_json = {"username": username, "password": password}
    login_req = requests.post(API_URL + 'auth/login', json=login_json)
    cookies = login_req.cookies
    return cookies

def download_file(req, task_id):
    with open(os.path.join(saved_dir, '{}.zip'.format(task_id)), 'wb') as f:
        for chunk in req.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("Task {}: download completed".format(task_id))

In [6]:
class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("LinhBD", "Aa@123456")  # type tuple
        self.server_host = "139.162.12.110"
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = "test"
        self.labels = [{"name":"text", "attributes": []}]  # type list
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = ["/home/phan.huy.hoang/Pictures/test_img_cvat.zip"]
        self.annotation_path = "/home/phan.huy.hoang/cvat/utils/cli/instances_default.json"
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

In [7]:
img_dir = "/home/doan.bao.linh/Downloads/data/kyc/OCR/images"

In [8]:
args = Args()
CVAT_URL = "http://{}:{}/".format(args.server_host, args.server_port)
API_URL = CVAT_URL + "api/v1/"
username, password = args.auth
username, password

('LinhBD', 'Aa@123456')

In [9]:
cookies = get_cookies()
cookies

<RequestsCookieJar[Cookie(version=0, name='csrftoken', value='UMMNTMAen1R6rFGFmGR5dq5xcfr4zjG1m1shLoGk3tJ8JQWQ3ZiIhqRcGOizxYvF', port=None, port_specified=False, domain='139.162.12.110', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1661758205, discard=False, comment=None, comment_url=None, rest={'SameSite': 'Lax'}, rfc2109=False), Cookie(version=0, name='sessionid', value='zj6mosz4l7k9b87sl1j3b1hotjvw0cya', port=None, port_specified=False, domain='139.162.12.110', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1631518205, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False)]>

In [10]:
req = requests.get("{}/api/v1/tasks/94".format(CVAT_URL), cookies=cookies)
req.text

'{"url":"http://139.162.12.110:8080/api/v1/tasks/94","id":94,"name":"[eKYC_DETECTOR]1628603190_cmnd_unknown","project_id":null,"mode":"annotation","owner":{"url":"http://139.162.12.110:8080/api/v1/users/34","id":34,"username":"admin","first_name":"","last_name":""},"assignee":null,"bug_tracker":"","created_date":"2021-08-10T13:46:31.108820Z","updated_date":"2021-08-21T02:29:21.913051Z","overlap":0,"segment_size":100,"status":"validation","labels":[{"id":1270,"name":"remove","color":"#ab8fb1","attributes":[{"id":1270,"name":"text_label","mutable":false,"input_type":"text","default_value":"","values":[""]}]},{"id":1269,"name":"right_fingerprint","color":"#b7350f","attributes":[{"id":1269,"name":"text_label","mutable":false,"input_type":"text","default_value":"","values":[""]}]},{"id":1268,"name":"left_fingerprint","color":"#14f8c0","attributes":[{"id":1268,"name":"text_label","mutable":false,"input_type":"text","default_value":"","values":[""]}]},{"id":1267,"name":"personal_identificatio

In [15]:
task_ids = list(range(94, 154))  # ignore task 1047
print(len(task_ids))

60


## Download ảnh + label

In [16]:
saved_dir = "/home/doan.bao.linh/Downloads/data/kyc/OCR/zip"
assert os.path.exists(saved_dir)
remove_all_content(saved_dir)

In [17]:
url_fmt = "http://{}:8080/api/v1/tasks/{}/dataset?format=COCO%201.0&action=download"

In [18]:
for task_id in tqdm_notebook(task_ids):
    url = url_fmt.format(args.server_host, task_id)

    while True:
        req = requests.get(url, cookies=cookies)
        if req.status_code == 200:
            download_file(req, task_id)
            break

    time.sleep(10)
    
# unzip all zip files in the same root folder
for task_id in task_ids:
    zip_fp = os.path.join(saved_dir, "{}.zip".format(task_id))
    assert os.path.exists(zip_fp)
    
    task_dir = os.path.join(saved_dir, str(task_id))
    if not os.path.exists(task_dir):
        os.mkdir(task_dir)
        
    with zipfile.ZipFile(zip_fp, "r") as zip_ref:
        zip_ref.extractall(task_dir)

/home/doan.bao.linh/miniconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/60 [00:00<?, ?it/s]

'Task 94: download completed'
'Task 95: download completed'
'Task 96: download completed'
'Task 97: download completed'
'Task 98: download completed'
'Task 99: download completed'
'Task 100: download completed'
'Task 101: download completed'
'Task 102: download completed'
'Task 103: download completed'
'Task 104: download completed'
'Task 105: download completed'
'Task 106: download completed'
'Task 107: download completed'
'Task 108: download completed'
'Task 109: download completed'
'Task 110: download completed'
'Task 111: download completed'
'Task 112: download completed'
'Task 113: download completed'
'Task 114: download completed'
'Task 115: download completed'
'Task 116: download completed'
'Task 117: download completed'
'Task 118: download completed'
'Task 119: download completed'
'Task 120: download completed'
'Task 121: download completed'
'Task 122: download completed'
'Task 123: download completed'
'Task 124: download completed'
'Task 125: download completed'
'Task 126: dow